In [1]:
import re
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
node = []
allDatas = []
siganlList = []
SignalsName = []
messageName = []

In [2]:
def readFile(file_name):
    global node,allDatas,siganlList,SignalsName,messageName
    ''' 得到dbc文件的绝对路径'''
    filePath = file_name
    if filePath:
        print(filePath)
        f = open(filePath, "r")  # 设置文件对象
    else:
        print("读取文件失败！")
        return 0
    """
    NodesPattern:节点
    MessagePattern：消息
    SignalPattern：信号
    """
    NodesPattern = re.compile(r"BU_: (.*)", re.S)
    MessagePattern = re.compile(r"BO_ (.*?) (.*?): (.*?) (.*)", re.S)
#     SignalPattern = re.compile('''SG_ (.*?) : (.*?)\|(.*?)@.*? \((.*?),(.*?)\) \[(.*?)\|(.*?)\] "(.*?)" (.*)''', re.S)
    SignalPattern = re.compile('''SG_ (.*?) : (.*?)\|(.*?)@([0-9])([+|-]) \((.*?),(.*?)\) \[(.*?)\|(.*?)\] "(.*?)" (.*)''', re.S)
    DefaultValue = '''BA_ "GenSig(.*?)" SG_ (\d+) signalname (\d+);'''

    line = f.readline()
    allDatas=[]
    while line:
        """ 匹配出节点 """
        NodesSearched = re.search(NodesPattern, line.strip())
        if NodesSearched:
            node = NodesSearched.group(1).split(" ")
            #print(node)
        """ 匹配出消息 """
        MessageSearched = re.search(MessagePattern, line.strip())
        if MessageSearched:
            siganlList.clear()
            """如果匹配到了message，则获取到message的相关参数 
             比如匹配到了NM_Message_ESC_409，则会解析出改message的一些参数构成list对象['1033', 'NM_Message_ESC_409', '8', 'ESC']
             这四个参数分别是 messgage ID ;message name ; messgae dataLenth ,message sender
             而且把这个list对象 加在了 siganlList 索引0的位置
            """
            Message = list(MessageSearched.groups())
            siganlList.append(Message)
            """ 只 要 message的名字 messageName 列表中"""
            messageName.append(Message[1])
            """读取下一行"""
            line = f.readline()
            """因为有些message并没有定义signal，所以 下一行还是message"""
            MessageSearched = re.search(MessagePattern, line.strip())
            SignalSearched = re.search(SignalPattern, line.strip())
            """下一行如果不是message的内容 就一定是signal的内容了"""
            if not MessageSearched:
                while SignalSearched:
                    """获取信号的参数追加到siganlList"""
                    signal = list(SignalSearched.groups())
                    siganlList.append(signal)
                    """只获取 signal name"""
                    SignalsName.append(signal[0])

                    # 再次解析信号，直到这个message下的信号全部解析完毕
                    line = f.readline()
                    SignalSearched = re.search(SignalPattern, line.strip())
           # print(siganlList)
            c = copy.deepcopy(siganlList)
            allDatas.append(c)
        else:
            line = f.readline()
            MessageSearched = re.search(MessagePattern, line.strip())
    f.close()  # 将文件关闭
    return 1

In [3]:
import os
file_names = []
# path = os.path.abspath('../My_DBC')
path = os.path.abspath('../opendbc-master/opendbc-master-delFalseData')
# path = os.path.abspath('./opendbc-master/opendbc-master')
dirs = os.listdir(path)                    # 获取指定路径下的文件
for i in dirs:
    if os.path.splitext(i)[1] == ".dbc":
        file_names.append(os.path.join(path,i))
# file_names

In [4]:
def get_id_labels(index):
    file_name = file_names[index]
    readFile(file_name)

    # 将DBC文件中的ID(10进制数)改为ID(16进制数)
    data_gts = {}
    for alldata in allDatas:
        data_gts[str(hex(int(alldata[0][0])))[2:]] = alldata

    # 得到DBC文件中各ID Message对应信号位置
    all_id_lsbs = {}
    for k in data_gts.keys():
        message = data_gts[k]
        lsb_list = []
        message_id = message[0][0]+"_"+message[0][1]
        del message[0]
        for signal in message:
            start = int(signal[1])
            length = int(signal[2])
            endianness = int(signal[3])
            start_row = int(start / 8)
            start_col = 8 - (start - start_row * 8) - 1
            new_start = start_row * 8 + start_col
            if endianness == 0:
                new_end = new_start + length
                cur_lsb = new_end - 1
            if endianness == 1:
                cur_lsb = new_start
            lsb_list.append(cur_lsb)
        all_id_lsbs[k] = lsb_list
    return all_id_lsbs

In [5]:
def get_signal_positions(index):
    file_name = file_names[index]
    readFile(file_name)
    
    data_gts = {}
    for alldata in allDatas:
        data_gts[str(hex(int(alldata[0][0])))[2:]] = alldata
        
    signal_positions = {}
    flag = False
    for k in data_gts.keys():
        message = data_gts[k]
        matrix = [[0] * 8 for _ in range(8)]
        message_id = message[0][0]+"_"+message[0][1]
        del message[0]
        cur_signal_positions = []
        for signal in message:
            start = int(signal[1])
            length = int(signal[2])
            endianness = int(signal[3])
            start_row = int(start / 8)
            start_col = 8 - (start - start_row * 8) - 1
            new_start = start_row * 8 + start_col
            if endianness == 0:
                new_end = new_start + length
                cur_msb = new_start
                cur_lsb = new_end - 1
            if endianness == 1:
                cur_lsb = new_start
                count = 0
                x = start_row
                y = start_col
                while count < (length-1):
                    if y == 0:
                        y = 7
                        x = x + 1
                    else:
                        y = y - 1
                    count = count + 1
                cur_msb = x * 8 + y
            cur_signal_positions.append((cur_msb, cur_lsb))
        signal_positions[k] = cur_signal_positions
    return signal_positions

In [6]:
def get_label01(all_id_lsbs):
    # 将各ID Message的信号位置改为 0/1 label
    all_id_labels = {}
    for k in all_id_lsbs:
        cur_lsb_list = all_id_lsbs[k]
        cur_label = np.zeros(64)
        for cur_lsb in cur_lsb_list:
            cur_label[cur_lsb] = 1
        all_id_labels[k] = cur_label
    return all_id_labels

In [7]:
def get_labels(all_id_lsbs):
    all_id_labels = {}
    for k in all_id_lsbs:
        cur_lsb_list = all_id_lsbs[k]
        cur_label = [2] * 64
        for cur_lsb in cur_lsb_list:
            cur_label[cur_lsb] = 1
        all_id_labels[k] = cur_label
    return all_id_labels

In [8]:
def get_trace(data):
    group = data.groupby(by="ID")
    id_tracedict = {}
    for trace in list(group):
        id_tracedict[trace[0]] = trace[1]["bin"].apply(lambda x: list(x)).apply(lambda x: list(map(int, x))).tolist()
    return id_tracedict

In [9]:
def get_ids(id_tracedict_int, all_id_labels):
    ids = []
    for id in id_tracedict_int.keys():
        if(id_tracedict_int.__contains__(id) and all_id_labels.__contains__(id)):
            ids.append(id)
    return ids

In [10]:
import random
# 设置全局的随机种子
random.seed(100)
# 以id为单位shuffle
def get_train_test_ids(id_tracedict, all_id_labels):
    ids = get_ids(id_tracedict, all_id_labels)
    random.shuffle(ids)
    train_len = int(len(ids)*0.9)
    train_ids = ids[:train_len]
    test_ids = ids[train_len:]
    return train_ids, test_ids

In [11]:
def get_trace_data(id_tracedict_int, all_id_labels): 
    cur_trace_data = []
    cur_trace_label = []
    for id in id_tracedict_int.keys():
        cur_trace = id_tracedict_int[id]
        cur_label = all_id_labels[id]
        length = len(cur_trace)
        start = 4000
        end = start + 1000
        n = int((length-4000)/1000)
        for i in range(n):
            cur_trace_data.append(cur_trace[start: end])
            cur_trace_label.append(cur_label)
            start = end
            end = start + 1000
    return cur_trace_data, cur_trace_label

In [12]:
def get_data_label(id_tracedict_int, all_id_labels):
    cur_trace_data, cur_trace_label = get_trace_data(id_tracedict_int, all_id_labels)
    trace_indexs = pd.DataFrame(cur_trace_label).index.tolist()
    random.shuffle(trace_indexs)
    trace_data, trace_labels = np.array(cur_trace_data)[trace_indexs], np.array(cur_trace_label)[trace_indexs]
    return trace_data, trace_labels

In [13]:
def get_trace_data_dict(cur_ids, id_tracedict_int, all_id_labels, trace_data_dict, trace_label_dict): 
    for id in cur_ids:
        trace_data_dict[id] = id_tracedict_int[id]
        trace_label_dict[id] = all_id_labels[id]

In [14]:
honda_dbcs = []
cadillac_dbcs = []
gm_dbcs = []
for i in range(len(file_names)):
    cur_name = file_names[i].split("\\")[-1].split("_")[0]
    if cur_name == "honda":
        honda_dbcs.append(i)
    if cur_name == "cadillac":
        cadillac_dbcs.append(i)
    if cur_name == "gm":
        gm_dbcs.append(i)

In [15]:
honda_id_labels = {}
for cur_dbc_index in honda_dbcs:
    honda_id_labels.update(get_labels(get_id_labels(cur_dbc_index))) 
cadillac_id_labels = {}
for cur_dbc_index in cadillac_dbcs:
    cadillac_id_labels.update(get_labels(get_id_labels(cur_dbc_index)))  
for cur_dbc_index in gm_dbcs:
    cadillac_id_labels.update(get_labels(get_id_labels(cur_dbc_index))) 

D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\honda_accord_2018_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\honda_civic_hatchback_ex_2017_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\honda_civic_sedan_16_diesel_2019_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\honda_civic_touring_2016_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\honda_clarity_hybrid_2018_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\honda_crv_executive_2016_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNet

In [16]:
data_hd = pd.read_csv('../Data/Honda/honda001.csv')
data_hd["bin"] = data_hd["Data"].apply(int, base=16).apply(bin).str[2:].apply(lambda x: x.zfill(64))
id_tracedict_hd = get_trace(data_hd)
train_ids_hd, test_ids_hd = get_train_test_ids(id_tracedict_hd, honda_id_labels)

In [17]:
data_cd = pd.read_csv('../Data/Cadillac/CadillacCSV/001.txt.csv')
data_cd["bin"] = data_cd["Data"].apply(int, base=16).apply(bin).str[2:].apply(lambda x: x.zfill(64))
id_tracedict_cd = get_trace(data_cd)
train_ids_cd, test_ids_cd = get_train_test_ids(id_tracedict_cd, cadillac_id_labels)

In [18]:
train_data_dict = {}
train_label_dict = {}
get_trace_data_dict(train_ids_hd, id_tracedict_hd, honda_id_labels, train_data_dict, train_label_dict)
get_trace_data_dict(train_ids_cd, id_tracedict_cd, cadillac_id_labels, train_data_dict, train_label_dict)

test_data_dict = {}
test_label_dict = {}
get_trace_data_dict(test_ids_hd, id_tracedict_hd, honda_id_labels, test_data_dict, test_label_dict)
get_trace_data_dict(test_ids_cd, id_tracedict_cd, cadillac_id_labels, test_data_dict, test_label_dict)

In [19]:
# 使用READ方法计算bit-flip
def get_bit_Flips(cur_id_traces):
    bit_Flips = []
    for i in range(len(cur_id_traces)):
        messagelist = cur_id_traces[i]
        trace_len = len(messagelist)
        bit_flip = np.zeros(64)
        previous = messagelist[0]
        for item in messagelist:
            for ix in range(64):
                if item[ix] != previous[ix]:
                    bit_flip[ix] = bit_flip[ix] + 1
            previous = item
        for ix in range(64):
            bit_flip[ix] = bit_flip[ix] / trace_len
        bit_flip = np.append(bit_flip, 0)
        bit_flip = np.append(bit_flip, 0)
        bit_Flips.append(bit_flip)
    return bit_Flips

In [20]:
# 先删除constant bit，然后，将trace序列拼成64。问题：会破环bit之间的相关性
def del_constant_bit_concate(cur_trace_data, cur_trace_label, bit_Flips):
    all_trace_data = []
    all_trace_label = []
    for i in range(len(bit_Flips)):
        cur_bit_flip = bit_Flips[i]
        cur_constant_bits = []
        for j in range(64):
            if cur_bit_flip[j] == 0:
                cur_constant_bits.append(j)
        cur_trace = []
        for temp in cur_trace_data[i]:
            cur_trace.append([temp[j] for j in range(len(temp)) if j not in cur_constant_bits])
        cur_label = []
        for j in range(len(cur_trace_label[i])):
            next_j = j+1
            if j not in cur_constant_bits:
                if next_j in cur_constant_bits:
                    cur_label.append(1)
                else:
                    cur_label.append(cur_trace_label[i][j])   
        if i == 0:
            all_trace_data = np.array(cur_trace)
            all_trace_label = np.array(cur_label)
        else:
            all_trace_data = np.hstack((all_trace_data, np.array(cur_trace)))
            all_trace_label = np.hstack((all_trace_label, np.array(cur_label)))
    
    
    num = int(all_trace_data.shape[1]/64)
    cur_trace_data = np.array(np.array_split(all_trace_data[:,0:num*64], num, axis=1))
    cur_trace_label = np.array(np.array_split(all_trace_label[0:num*64], num, axis=0))
    return cur_trace_data, cur_trace_label

In [21]:
def fill_list(cur_list, number, length):
    for i in range(len(cur_list),length):
        cur_list.append(number)
    return cur_list

In [22]:
# 只删除constant bit，形成不定长的trace 序列
def del_constant_bit(cur_trace_data, cur_trace_label, bit_Flips):
    all_trace_data = []
    all_trace_label = []
    for i in range(len(bit_Flips)):
        cur_bit_flip = bit_Flips[i]
        cur_constant_bits = []
        for j in range(64):
            if cur_bit_flip[j] == 0:
                cur_constant_bits.append(j)
        cur_trace = []
        for temp in cur_trace_data[i]:
            res_trace = [temp[j] for j in range(len(temp)) if j not in cur_constant_bits]
            cur_trace.append(fill_list(res_trace, -1, 64))
        cur_label = []
        for j in range(len(cur_trace_label[i])):
            next_j = j+1
            if j not in cur_constant_bits:
                if next_j in cur_constant_bits:
                    cur_label.append(1)
                else:
                    cur_label.append(cur_trace_label[i][j])  
        cur_label = fill_list(cur_label, -1, 64)
        all_trace_data.append(cur_trace)
        all_trace_label.append(cur_label)
    return np.array(all_trace_data), np.array(all_trace_label)

In [23]:
# 只删除constant bit，形成不定长的trace 序列
def del_constant_bit_new(cur_trace_data, cur_trace_label, bit_Flips):
    all_trace_data = []
    all_trace_label = []
    for i in range(len(bit_Flips)):
        cur_bit_flip = bit_Flips[i]
        cur_constant_bits = []
        for j in range(64):
            if cur_bit_flip[j] == 0:
                if j == 0 or cur_bit_flip[j-1] == 0:
                    cur_constant_bits.append(j)
        cur_trace = []
        for temp in cur_trace_data[i]:
            res_trace = [temp[j] for j in range(len(temp)) if j not in cur_constant_bits]
            cur_trace.append(fill_list(res_trace, -1, 64))
        cur_label = []
        for j in range(len(cur_trace_label[i])):
            next_j = j+1
            if j not in cur_constant_bits:
                if cur_bit_flip[j] != 0 and cur_bit_flip[next_j] == 0:
                    cur_label.append(1)
                else:
                    cur_label.append(cur_trace_label[i][j])  
        cur_label = fill_list(cur_label, -1, 64)
        all_trace_data.append(cur_trace)
        all_trace_label.append(cur_label)
    return np.array(all_trace_data), np.array(all_trace_label)

In [27]:
def change_labels(cur_trace_data, cur_trace_label, bit_Flips):
    for i in range(len(bit_Flips)):
        cur_bit_flip = bit_Flips[i]
        for j in range(64):
            if cur_bit_flip[j] == 0:
                cur_trace_label[i][j] = 0
    return cur_trace_data, cur_trace_label

In [25]:
# shuffle后得到训练集和测试集，其中训练集的id占70%，测试集的id占30%
train_data, train_labels = get_data_label(train_data_dict, train_label_dict)
bit_Flips_train = get_bit_Flips(train_data)

In [28]:
train_data, train_labels = change_labels(train_data, train_labels, bit_Flips_train)
# train_data_del, train_labels_del = del_constant_bit(train_data, train_labels, bit_Flips_train)
# train_data_del_new, train_labels_del_new = del_constant_bit_new(train_data, train_labels, bit_Flips_train)

In [29]:
test_data, test_labels = get_data_label(test_data_dict, test_label_dict)
bit_Flips_test = get_bit_Flips(test_data)

In [30]:
test_data, test_labels = change_labels(test_data, test_labels, bit_Flips_test)
# test_data_del, test_labels_del = del_constant_bit(test_data, test_labels, bit_Flips_test)
# test_data_del_new, test_labels_del_new = del_constant_bit_new(test_data, test_labels, bit_Flips_test)

In [31]:
# train_data_del_lstm = train_data_del.transpose(0,2,1)
# test_data_del_lstm = test_data_del.transpose(0,2,1)

In [32]:
train_data_lstm = train_data.transpose(0,2,1)
test_data_lstm = test_data.transpose(0,2,1)

In [ ]:
def cal_score(model, train_data, train_labels):
    print("数据集:")
    predict_target_msb_prob = model.predict(train_data)
    predict_target_msb_label = (predict_target_msb_prob > 0.5).astype(int)
    predict_target_msb_1D = predict_target_msb_label.flatten()
    train_labels_1D = train_labels.flatten()
    print("预测正确数量,训练集样本量:")
    print(sum(predict_target_msb_1D == train_labels_1D),len(train_labels_1D))
    print("训练集精确度等指标：")
    print(metrics.classification_report(train_labels_1D,predict_target_msb_1D))
    print("混淆矩阵：")
    print(metrics.confusion_matrix(train_labels_1D,predict_target_msb_1D))